### User input

In [ ]:
decimal = '.'                                                                   # decimal of the input file
sep = ';'                                                                       # separator of the input file

from config import *                                                            # Personal settings of local user to set input and output directories
input_dir = input_directory + 'results/df_mediancor_sw_30.0.csv'                # input directory
output = output_directory + 'results/'                                          # output directory

dateheadername = 'Timestamp'                                                    # header of the date  
header = 0                                                                      # header row number
dateparsingformat = '%Y-%m-%d %H:%M:%S'                                         # format of the date 
sample_name = 'sw'                                                              # name of the sample

timestart = '2018-11-13 05:02:00'                                               # starting date
timeend = '2018-12-04 08:44:00'                                                 # ending date
wv1 = '220 nm'                                                                  # wavelength numerator
wv2 = '255 nm'                                                                  # wavelength denominator

# To plot the time series of ratio of absorbance data, the user can modify:
title1 = 'absorbance_ratio' + str(wv1) + '_' + str(wv2)                         # title of the exported figure 1
ylabel = 'A$_{220}$/A$_{255}$ [dimensionless]'                                  # label y-axis
fig_format = '.tiff'                                                            # format of the exported figure
dpi = 300                                                                       # resolution of the exported figure

# To plot the time series of changes (%) ratio of absorbance data, the user can modify:
date_ref_start = '2018-11-13 05:02:00'                                          # define reference period for computing the changes
date_ref_end = '2018-11-14 05:02:00'
ylabel2 = 'A$_{220}$/A$_{255}$ change [%]'                                      # label y-axis
title2 = 'absorbance_ratio_change_' + str(wv1) + '_' + str(wv2)                 # title of the exported figure 2

# To look at the change in percentage at a certain date:
date_interesting = '2018-12-04 08:38:00'                                        # specify the date of interest

### Start environment and import data

In [ ]:
import abspectroscopy_functions as abspy # Functions from the AbspectroscoPY toolbox
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)

df = pd.read_csv(input_dir, sep = sep, header = header, index_col = 0) 
df.index = pd.to_datetime(df.index)      # make sure time column (here index) is using time format
df

### abs_ratio

In [ ]:
def abs_ratio(df_in, 
              date_ref_start,
              date_ref_end,
              dateparsingformat,
              wavelength1,
              wavelength2,              
              date_string):
    '''
    function to calculate the ratio of absorbance data at two different wavelengths and compute the percentage change in relation to a reference period 
    :argument df_in: dataframe in input 
    :argument date_ref_start: starting date of the reference period
    :argument date_ref_end: ending date of the reference period
    :argument dateparsingformat: format of the dates
    :argument wavelength1: wavelength numerator
    :argument wavelength2: wavelength denominator
    :argument date_string: date where to look at the percentage change compared to the reference period
    :return: dataframe with the ratio of absorbance data at two different wavelengths (df_out1) and with percentage changes of the ratio of absorbance data in relation to a reference period (df_out2)
    '''    
    df_ratio = df_in[wavelength1]/df_in[wavelength2]
    
    ### COMPUTE THE PERCENTAGE CHANGES OF THE ABSORBANCE RATIOS DATA IN RELATION TO A REFERENCE PERIOD AND PLOT IT
    # Define a reference period:
    date_ref_start1 = str(df_in.iloc[df_in.index.get_loc(date_ref_start, method='nearest')].name) # find the closest date to the date_ref_start and the second closest date to the date_ref_end available in the dataframe
    date_ref_end1 = str(df_in.iloc[df_in.index.get_loc(date_ref_end, method='nearest')+1].name)
    timestamp_selected = pd.date_range(start = date_ref_start1, end = date_ref_end1, periods = 2).tolist()    
    idx_vector = []
    for timestamp in timestamp_selected:
            index_date_time=np.where(df_in.index == timestamp.strftime(dateparsingformat))[0]     # to check for specific date-time
            idx=index_date_time[:][0]
            idx_vector.append(idx)
    #print('idx_vector:',idx_vector)

    df_ref = df_ratio.copy()
    df_ref = df_ref[idx_vector[0]:idx_vector[1]]

    # Normalize the data by the average of the reference period and multiply by 100 to get %change:
    df_ref_av = df_ref.mean()                                                                     # compute the average of the vector
    df_sub_ref_av = df_ratio.copy()
    df_sub_ref_av = df_sub_ref_av - df_ref_av                                                     # subtract this average to the vector
    df_change_per = (df_sub_ref_av/abs(df_ref_av))*100                                            # compute the percent change

    # Exclude from the dataset the reference period:
    df_change_per = df_change_per.drop(df_change_per.index[idx_vector[0]:idx_vector[1]])
    
    ### PERCENTAGE CHANGE AT A SPECIFIC DATE:
    print('Percentage change at the specified date:', df_change_per.iloc[np.where(df_change_per.index == date_string)])
    
    df_out1 = df_ratio
    df_out2 = df_change_per
    return(df_out1, df_out2)

In [ ]:
df_ratio, df_change_per = abs_ratio(df, date_ref_start, date_ref_end, dateparsingformat, wv1, wv2, date_interesting)
df_change_per.to_csv(output + 'df_absorbance_ratio_change_' + str(wv1) + '_' + str(wv2) + '.csv', index = True) # export the dataframe

### Visualise the absorbance ratio time series

In [ ]:
%matplotlib inline # necessary if the notebook is not configured to use the inline backend by default
#%matplotlib notebook
plt.ion()
abspy.makeaplot_nocolsel(df_change_per, output, timestart, timeend, wv1, wv2, sample_name, title1) 

### Visualise the variation of absorbance ratios in relation to a reference period in terms of percentage changes

In [ ]:
#%matplotlib inline 
%matplotlib notebook
plt.ion()
abspy.makeaplot_nocolsel(df_change_per, output, timestart, timeend, wv1, wv2, sample_name, title2, ylabel = '{} {} '.format('A' + '$_{' + wv1 + '}$' + '/' + 'A' + '$_{' + wv2 + '}$', 'change [%]')) 